In [1]:
import pandas as pd
import copy
import matplotlib.pyplot as plt
import math
import numpy as np
import seaborn as sns
import tqdm.notebook as tq
import scipy.cluster.hierarchy as spc
from scipy.cluster.hierarchy import dendrogram, linkage
from scipy.cluster.hierarchy import fcluster
from sklearn.feature_selection import VarianceThreshold
from tqdm import tqdm
from collections import Counter
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import train_test_split as tts, cross_val_score as cv, RepeatedStratifiedKFold as rsk
from sklearn.ensemble import RandomForestClassifier as rf, ExtraTreesClassifier as et, BaggingClassifier as bc
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, f1_score
from sklearn.utils import class_weight
import lightgbm as lgb

# EDA 

In [2]:
test_df = pd.read_csv("test.csv")

In [3]:
test_df.head()

,Id,v2a1,hacdor,rooms,hacapo,v14a,refrig,v18q,v18q1,r4h1,...,age,SQBescolari,SQBage,SQBhogar_total,SQBedjefe,SQBhogar_nin,SQBovercrowding,SQBdependency,SQBmeaned,agesq
0,ID_2f6873615,NaN,0,5,0,1,1,0,NaN,1,...,4,0,16,9,0,1,2.25,0.25,272.25,16
1,ID_1c78846d2,NaN,0,5,0,1,1,0,NaN,1,...,41,256,1681,9,0,1,2.25,0.25,272.25,1681
2,ID_e5442cf6a,NaN,0,5,0,1,1,0,NaN,1,...,41,289,1681,9,0,1,2.25,0.25,272.25,1681
3,ID_a8db26a79,NaN,0,14,0,1,1,1,1.0,0,...,59,256,3481,1,256,0,1.00,0.00,256.00,3481
4,ID_a62966799,175000.0,0,4,0,1,1,1,1.0,0,...,18,121,324,1,0,1,0.25,64.00,NaN,324


In [5]:
feature_description = pd.read_csv("codebook.csv").set_index("Variable name")

### Replacing variable names with description for greater understandability 

In [6]:
column_headers = list(test_df.columns)
#to move household identifier to the front
column_headers.remove('idhogar')
column_headers_original = ['idhogar'] + column_headers

test_df = test_df.loc[:,column_headers_original]
column_headers_original.remove('Id')
# column_headers_original.remove('Target')
column_headers_new = list(feature_description.loc[column_headers_original]['Variable description'])
column_headers_new.insert(1,'Id')
# column_headers_new.append('Target')

In [7]:
test_df.columns = column_headers_new
# column_headers_new

In [8]:

test_df.head()

,Household level identifier,Id,Monthly rent payment,=1 Overcrowding by bedrooms,number of all rooms in the house,=1 Overcrowding by rooms,=1 has toilet in the household,=1 if the household has refrigerator,owns a tablet,number of tablets household owns,...,Age in years,escolari squared,age squared,hogar_total squared,edjefe squared,hogar_nin squared,overcrowding squared,dependency squared,meaned squared,Age squared
0,72958b30c,ID_2f6873615,NaN,0,5,0,1,1,0,NaN,...,4,0,16,9,0,1,2.25,0.25,272.25,16
1,72958b30c,ID_1c78846d2,NaN,0,5,0,1,1,0,NaN,...,41,256,1681,9,0,1,2.25,0.25,272.25,1681
2,72958b30c,ID_e5442cf6a,NaN,0,5,0,1,1,0,NaN,...,41,289,1681,9,0,1,2.25,0.25,272.25,1681
3,5b598fbc9,ID_a8db26a79,NaN,0,14,0,1,1,1,1.0,...,59,256,3481,1,256,0,1.00,0.00,256.00,3481
4,1e2fc704e,ID_a62966799,175000.0,0,4,0,1,1,1,1.0,...,18,121,324,1,0,1,0.25,64.00,NaN,324


In [9]:
# test_df.to_csv('test_df_renamed.csv', index=False)

### Filling nan values

In [10]:
test_df.loc[:, test_df.isna().any()].head()

,Monthly rent payment,number of tablets household owns,Years behind in school,average years of education for adults (18+),meaned squared
0,NaN,NaN,NaN,16.5,272.25
1,NaN,NaN,NaN,16.5,272.25
2,NaN,NaN,NaN,16.5,272.25
3,NaN,1.0,NaN,16.0,256.00
4,175000.0,1.0,NaN,NaN,NaN


#### 	Monthly rent payment	

In [11]:
test_df[test_df["Monthly rent payment"].isna()]

,Household level identifier,Id,Monthly rent payment,=1 Overcrowding by bedrooms,number of all rooms in the house,=1 Overcrowding by rooms,=1 has toilet in the household,=1 if the household has refrigerator,owns a tablet,number of tablets household owns,...,Age in years,escolari squared,age squared,hogar_total squared,edjefe squared,hogar_nin squared,overcrowding squared,dependency squared,meaned squared,Age squared
0,72958b30c,ID_2f6873615,NaN,0,5,0,1,1,0,NaN,...,4,0,16,9,0,1,2.250000,0.25,272.2500,16
1,72958b30c,ID_1c78846d2,NaN,0,5,0,1,1,0,NaN,...,41,256,1681,9,0,1,2.250000,0.25,272.2500,1681
2,72958b30c,ID_e5442cf6a,NaN,0,5,0,1,1,0,NaN,...,41,289,1681,9,0,1,2.250000,0.25,272.2500,1681
3,5b598fbc9,ID_a8db26a79,NaN,0,14,0,1,1,1,1.0,...,59,256,3481,1,256,0,1.000000,0.00,256.0000,3481
9,b61e2bb92,ID_24864adcc,NaN,0,6,0,1,1,0,NaN,...,70,100,4900,1,0,0,0.111111,64.00,100.0000,4900
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23851,3aa78c56b,ID_a065a7cad,NaN,1,2,1,1,1,0,NaN,...,10,9,100,36,25,4,36.000000,0.25,33.0625,100
23852,d237404b6,ID_1a7c6953b,NaN,0,3,0,1,1,0,NaN,...,54,36,2916,16,36,4,4.000000,1.00,36.0000,2916
23853,d237404b6,ID_07dbb4be2,NaN,0,3,0,1,1,0,NaN,...,12,16,144,16,36,4,4.000000,1.00,36.0000,144
23854,d237404b6,ID_34d2ed046,NaN,0,3,0,1,1,0,NaN,...,12,25,144,16,36,4,4.000000,1.00,36.0000,144


In [12]:
["=1 own and fully paid house", "=1 own, paying in installments", "=1 rented", "=1 precarious", "=1 other(assigned, borrowed)"]

['=1 own and fully paid house',
 '=1 own, paying in installments',
 '=1 rented',
 '=1 precarious',
 '=1 other(assigned, borrowed)']

In [13]:
test_df.loc[test_df[test_df["=1 own and fully paid house"]==1].index,"Monthly rent payment"]  = test_df[test_df["=1 own and fully paid house"]==1]["Monthly rent payment"].fillna(0.0)

Not paying rent as they own the house

In [14]:
test_df[test_df["=1 own and fully paid house"]==1]

,Household level identifier,Id,Monthly rent payment,=1 Overcrowding by bedrooms,number of all rooms in the house,=1 Overcrowding by rooms,=1 has toilet in the household,=1 if the household has refrigerator,owns a tablet,number of tablets household owns,...,Age in years,escolari squared,age squared,hogar_total squared,edjefe squared,hogar_nin squared,overcrowding squared,dependency squared,meaned squared,Age squared
0,72958b30c,ID_2f6873615,0.0,0,5,0,1,1,0,NaN,...,4,0,16,9,0,1,2.250000,0.25,272.2500,16
1,72958b30c,ID_1c78846d2,0.0,0,5,0,1,1,0,NaN,...,41,256,1681,9,0,1,2.250000,0.25,272.2500,1681
2,72958b30c,ID_e5442cf6a,0.0,0,5,0,1,1,0,NaN,...,41,289,1681,9,0,1,2.250000,0.25,272.2500,1681
3,5b598fbc9,ID_a8db26a79,0.0,0,14,0,1,1,1,1.0,...,59,256,3481,1,256,0,1.000000,0.00,256.0000,3481
9,b61e2bb92,ID_24864adcc,0.0,0,6,0,1,1,0,NaN,...,70,100,4900,1,0,0,0.111111,64.00,100.0000,4900
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23851,3aa78c56b,ID_a065a7cad,0.0,1,2,1,1,1,0,NaN,...,10,9,100,36,25,4,36.000000,0.25,33.0625,100
23852,d237404b6,ID_1a7c6953b,0.0,0,3,0,1,1,0,NaN,...,54,36,2916,16,36,4,4.000000,1.00,36.0000,2916
23853,d237404b6,ID_07dbb4be2,0.0,0,3,0,1,1,0,NaN,...,12,16,144,16,36,4,4.000000,1.00,36.0000,144
23854,d237404b6,ID_34d2ed046,0.0,0,3,0,1,1,0,NaN,...,12,25,144,16,36,4,4.000000,1.00,36.0000,144


=1 own, paying in installments

In [15]:
test_df[test_df['=1 own, paying in installments']==1].isna().any()

Household level identifier           False
Id                                   False
Monthly rent payment                 False
=1 Overcrowding by bedrooms          False
 number of all rooms in the house    False
                                     ...  
hogar_nin squared                    False
overcrowding squared                 False
dependency squared                   False
meaned squared                       False
Age squared                          False
Length: 142, dtype: bool

=1 rented

In [16]:
test_df[test_df['=1 rented']==1].isna().any()

Household level identifier           False
Id                                   False
Monthly rent payment                 False
=1 Overcrowding by bedrooms          False
 number of all rooms in the house    False
                                     ...  
hogar_nin squared                    False
overcrowding squared                 False
dependency squared                   False
meaned squared                        True
Age squared                          False
Length: 142, dtype: bool

=1 precarious

In [17]:
test_df[test_df['=1 precarious']==1]

,Household level identifier,Id,Monthly rent payment,=1 Overcrowding by bedrooms,number of all rooms in the house,=1 Overcrowding by rooms,=1 has toilet in the household,=1 if the household has refrigerator,owns a tablet,number of tablets household owns,...,Age in years,escolari squared,age squared,hogar_total squared,edjefe squared,hogar_nin squared,overcrowding squared,dependency squared,meaned squared,Age squared
410,08b75efed,ID_ebe82ba02,NaN,0,3,0,1,1,0,NaN,...,0,0,0,9,0,4,2.25,4.0,25.0,0
411,08b75efed,ID_63daea6de,NaN,0,3,0,1,1,0,NaN,...,41,25,1681,9,0,4,2.25,4.0,25.0,1681
412,08b75efed,ID_643f77b72,NaN,0,3,0,1,1,0,NaN,...,14,49,196,9,0,4,2.25,4.0,25.0,196
569,37f3f2818,ID_9cb4e224d,NaN,0,4,0,1,1,1,1.0,...,43,64,1849,16,36,0,4.00,0.0,64.0,1849
570,37f3f2818,ID_4bd275816,NaN,0,4,0,1,1,1,1.0,...,27,121,729,16,36,0,4.00,0.0,64.0,729
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22923,25843c28f,ID_96e81928b,NaN,0,6,0,1,1,0,NaN,...,15,64,225,36,121,16,4.00,4.0,196.0,225
22924,25843c28f,ID_a1d72ddec,NaN,0,6,0,1,1,0,NaN,...,14,25,196,36,121,16,4.00,4.0,196.0,196
22925,25843c28f,ID_8c24e3b08,NaN,0,6,0,1,1,0,NaN,...,18,121,324,36,121,16,4.00,4.0,196.0,324
22926,25843c28f,ID_0e8b2f2e5,NaN,0,6,0,1,1,0,NaN,...,13,49,169,36,121,16,4.00,4.0,196.0,169


Assigned and Borrowed

In [18]:
test_df[test_df['=1 other(assigned, borrowed)']==1]

,Household level identifier,Id,Monthly rent payment,=1 Overcrowding by bedrooms,number of all rooms in the house,=1 Overcrowding by rooms,=1 has toilet in the household,=1 if the household has refrigerator,owns a tablet,number of tablets household owns,...,Age in years,escolari squared,age squared,hogar_total squared,edjefe squared,hogar_nin squared,overcrowding squared,dependency squared,meaned squared,Age squared
22,d5a0a19d8,ID_8be4c9bbf,NaN,0,5,0,1,1,0,NaN,...,25,169,625,4,0,0,1.00,1.00,121.000000,625
23,d5a0a19d8,ID_7bade887b,NaN,0,5,0,1,1,0,NaN,...,66,81,4356,4,0,0,1.00,1.00,121.000000,4356
28,8f4413ade,ID_1615bc9ef,NaN,0,8,0,1,1,1,1.0,...,27,49,729,9,64,0,1.44,0.25,75.111115,729
29,8f4413ade,ID_3bb0b62f1,NaN,0,8,0,1,1,1,1.0,...,78,64,6084,9,64,0,1.44,0.25,75.111115,6084
30,8f4413ade,ID_033b1af6b,NaN,0,8,0,1,1,1,1.0,...,62,121,3844,9,64,0,1.44,0.25,75.111115,3844
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23838,935a65ffa,ID_2789c94fa,NaN,0,4,0,1,1,0,NaN,...,1,0,1,36,16,16,9.00,4.00,25.000000,1
23839,935a65ffa,ID_da28a4a6b,NaN,0,4,0,1,1,0,NaN,...,9,4,81,36,16,16,9.00,4.00,25.000000,81
23840,935a65ffa,ID_35185fb42,NaN,0,4,0,1,1,0,NaN,...,39,36,1521,36,16,16,9.00,4.00,25.000000,1521
23841,935a65ffa,ID_19c0b1480,NaN,0,4,0,1,1,0,NaN,...,42,16,1764,36,16,16,9.00,4.00,25.000000,1764


In [19]:
test_df["Monthly rent payment"]  = test_df["Monthly rent payment"].fillna(0.0)

In [20]:
test_df['Monthly rent payment'].isna().sum()

0

In [21]:
test_df

,Household level identifier,Id,Monthly rent payment,=1 Overcrowding by bedrooms,number of all rooms in the house,=1 Overcrowding by rooms,=1 has toilet in the household,=1 if the household has refrigerator,owns a tablet,number of tablets household owns,...,Age in years,escolari squared,age squared,hogar_total squared,edjefe squared,hogar_nin squared,overcrowding squared,dependency squared,meaned squared,Age squared
0,72958b30c,ID_2f6873615,0.0,0,5,0,1,1,0,NaN,...,4,0,16,9,0,1,2.25,0.25,272.2500,16
1,72958b30c,ID_1c78846d2,0.0,0,5,0,1,1,0,NaN,...,41,256,1681,9,0,1,2.25,0.25,272.2500,1681
2,72958b30c,ID_e5442cf6a,0.0,0,5,0,1,1,0,NaN,...,41,289,1681,9,0,1,2.25,0.25,272.2500,1681
3,5b598fbc9,ID_a8db26a79,0.0,0,14,0,1,1,1,1.0,...,59,256,3481,1,256,0,1.00,0.00,256.0000,3481
4,1e2fc704e,ID_a62966799,175000.0,0,4,0,1,1,1,1.0,...,18,121,324,1,0,1,0.25,64.00,NaN,324
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23851,3aa78c56b,ID_a065a7cad,0.0,1,2,1,1,1,0,NaN,...,10,9,100,36,25,4,36.00,0.25,33.0625,100
23852,d237404b6,ID_1a7c6953b,0.0,0,3,0,1,1,0,NaN,...,54,36,2916,16,36,4,4.00,1.00,36.0000,2916
23853,d237404b6,ID_07dbb4be2,0.0,0,3,0,1,1,0,NaN,...,12,16,144,16,36,4,4.00,1.00,36.0000,144
23854,d237404b6,ID_34d2ed046,0.0,0,3,0,1,1,0,NaN,...,12,25,144,16,36,4,4.00,1.00,36.0000,144


In [22]:
test_df.loc[:, test_df.isna().any()].head()

,number of tablets household owns,Years behind in school,average years of education for adults (18+),meaned squared
0,NaN,NaN,16.5,272.25
1,NaN,NaN,16.5,272.25
2,NaN,NaN,16.5,272.25
3,1.0,NaN,16.0,256.00
4,1.0,NaN,NaN,NaN


In [23]:
# test_df["Monthly rent payment"] = test_df["Monthly rent payment"].dropna()

#### Number of tablets owned

In [24]:
test_df.loc[:, test_df.isna().any()]['number of tablets household owns'].isna().value_counts()

True     18126
False     5730
Name: number of tablets household owns, dtype: int64

In [25]:
test_df.loc[:, test_df.isna().any()]['number of tablets household owns'].value_counts()

1.0    4297
2.0    1005
3.0     318
4.0      88
5.0      18
6.0       4
Name: number of tablets household owns, dtype: int64

In [26]:
test_df['number of tablets household owns'] =  test_df['number of tablets household owns'].fillna(0.0)
test_df['number of tablets household owns'].head()

0    0.0
1    0.0
2    0.0
3    1.0
4    1.0
Name: number of tablets household owns, dtype: float64

#### Years behind in school

In [28]:
test_df.loc[:, test_df.isna().any()]['Years behind in school'].isna().value_counts()

True     19653
False     4203
Name: Years behind in school, dtype: int64

In [29]:
test_df.loc[:, test_df.isna().any()]['Years behind in school'].unique()

array([nan,  0.,  3.,  2.,  1.,  4.,  5., 99.])

In [30]:
# test_df = test_df.drop(columns=['Years behind in school'])

In [31]:
test_df['Years behind in school'] =  test_df['Years behind in school'].fillna(0)
# test_df['Years behind in school'].head()

#### average years of education for adults (18+)	

In [32]:
set(test_df.loc[:, test_df.isna().any()]["average years of education for adults (18+)"].unique())

{nan,
 0.0,
 0.33333334,
 0.5,
 0.66666669,
 0.75,
 1.0,
 1.25,
 1.3333334,
 1.5,
 1.6666666,
 1.75,
 1.8,
 2.0,
 2.2,
 2.25,
 2.3333333,
 2.5,
 2.5999999,
 2.6666667,
 2.75,
 3.0,
 3.2,
 3.25,
 3.3333333,
 3.5,
 3.6666667,
 3.75,
 3.8,
 4.0,
 4.1666665,
 4.1999998,
 4.25,
 4.3333335,
 4.5,
 4.5999999,
 4.6666665,
 4.75,
 4.8000002,
 5.0,
 5.25,
 5.3333335,
 5.4000001,
 5.5,
 5.5999999,
 5.6666665,
 5.7142859,
 5.75,
 5.8000002,
 5.8571429,
 6.0,
 6.1428571,
 6.1666665,
 6.1999998,
 6.25,
 6.3333335,
 6.4000001,
 6.4285712,
 6.5,
 6.599999900000001,
 6.666666499999999,
 6.75,
 6.8000002,
 7.0,
 7.166666499999999,
 7.1999998,
 7.25,
 7.333333500000001,
 7.400000099999999,
 7.4285712,
 7.5,
 7.599999900000001,
 7.666666499999999,
 7.75,
 7.8000002,
 8.0,
 8.199999799999999,
 8.25,
 8.333333,
 8.3999996,
 8.5,
 8.6000004,
 8.666667,
 8.75,
 8.8000002,
 9.0,
 9.1999998,
 9.25,
 9.2857141,
 9.333333,
 9.3999996,
 9.4285717,
 9.5,
 9.6000004,
 9.666667,
 9.75,
 9.8000002,
 9.833333,
 9.85714

In [33]:
test_df[test_df["average years of education for adults (18+)"].isna()][["average years of education for adults (18+)",'Age in years','years of schooling']]

,average years of education for adults (18+),Age in years,years of schooling
4,NaN,18,11
535,NaN,18,11
536,NaN,19,12
537,NaN,19,12
2612,NaN,19,8
2613,NaN,17,6
6809,NaN,18,12
6810,NaN,19,12
7266,NaN,19,12
7267,NaN,18,10


In [34]:
error_index = test_df[test_df["average years of education for adults (18+)"].isna()][['Age in years','years of schooling']].index
for index in error_index:
    test_df["average years of education for adults (18+)"][index] = test_df.iloc[index,:]['years of schooling'] 

/var/folders/fj/pkqhb62j0v35sdfyvpr9kd940000gn/T/ipykernel_97690/123202182.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_df["average years of education for adults (18+)"][index] = test_df.iloc[index,:]['years of schooling']


In [35]:
# test_df[test_df['Target']==4]["average years of education for adults (18+)"].mean()
# test_df["average years of education for adults (18+)"] =  test_df["average years of education for adults (18+)"].fillna(test_df[test_df['Target']==4]["average years of education for adults (18+)"].mean())
# test_df["average years of education for adults (18+)"].isna().sum()


#### meaned squared

In [36]:
test_df[test_df["meaned squared"].isna()]

,Household level identifier,Id,Monthly rent payment,=1 Overcrowding by bedrooms,number of all rooms in the house,=1 Overcrowding by rooms,=1 has toilet in the household,=1 if the household has refrigerator,owns a tablet,number of tablets household owns,...,Age in years,escolari squared,age squared,hogar_total squared,edjefe squared,hogar_nin squared,overcrowding squared,dependency squared,meaned squared,Age squared
4,1e2fc704e,ID_a62966799,175000.0,0,4,0,1,1,1,1.0,...,18,121,324,1,0,1,0.250000,64.0,NaN,324
535,bb7038e41,ID_ab0fe3b38,399378.0,0,6,0,1,0,1,3.0,...,18,121,324,1,0,1,1.000000,64.0,NaN,324
536,4937daf94,ID_4f297efe7,180000.0,0,4,0,1,1,0,0.0,...,19,144,361,4,0,4,1.000000,64.0,NaN,361
537,4937daf94,ID_b41dee3bb,180000.0,0,4,0,1,1,0,0.0,...,19,144,361,4,0,4,1.000000,64.0,NaN,361
2612,4deb0c65e,ID_ad3389376,100000.0,0,2,0,1,1,0,0.0,...,19,64,361,4,64,4,4.000000,64.0,NaN,361
2613,4deb0c65e,ID_2586969c3,100000.0,0,2,0,1,1,0,0.0,...,17,36,289,4,64,4,4.000000,64.0,NaN,289
6809,74718009d,ID_fd724fb68,135000.0,0,3,0,1,1,0,0.0,...,18,144,324,4,0,4,1.000000,64.0,NaN,324
6810,74718009d,ID_7f6900abb,135000.0,0,3,0,1,1,0,0.0,...,19,144,361,4,0,4,1.000000,64.0,NaN,361
7266,8e4c4b74b,ID_84ddecd60,90000.0,0,3,0,1,1,0,0.0,...,19,144,361,4,144,4,4.000000,64.0,NaN,361
7267,8e4c4b74b,ID_871fe4f99,90000.0,0,3,0,1,1,0,0.0,...,18,100,324,4,144,4,4.000000,64.0,NaN,324


In [37]:
error_index = test_df[test_df["meaned squared"].isna()][['Age in years','years of schooling']].index
for index in error_index:
    test_df["meaned squared"][index] = (test_df.iloc[index,:]['years of schooling'])**2

/var/folders/fj/pkqhb62j0v35sdfyvpr9kd940000gn/T/ipykernel_97690/2675917202.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_df["meaned squared"][index] = (test_df.iloc[index,:]['years of schooling'])**2


### Sorting "Yes" aand "No"

In [38]:
yes_no_columns = []
for i in range(len(test_df.columns)):
    value_list = test_df.iloc[:,i].unique()
    if 'yes' in value_list or 'no' in value_list:
        yes_no_columns.append(test_df.iloc[:,i].name)

/var/folders/fj/pkqhb62j0v35sdfyvpr9kd940000gn/T/ipykernel_97690/3649530830.py:4: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if 'yes' in value_list or 'no' in value_list:


In [39]:
yes_no_columns

['Dependency rate',
 'years of education of male head of household',
 'years of education of female head of household']

#### Dependecy

In [40]:
test_df['Dependency rate'].value_counts()

yes          5388
no           4289
.5           3678
2            1769
1.5          1758
.33333334    1533
.66666669    1130
8            1037
.25           684
3             596
1.3333334     278
2.5           224
.2            216
.75           203
4             195
.40000001     175
.60000002     128
1.6666666     120
5              96
.16666667      56
1.25           54
.80000001      45
.14285715      32
2.3333333      30
.83333331      22
3.5            18
7              16
3.3333333      13
.85714287      13
2.25           13
.375           11
1.2            11
.2857143        9
.125            9
6               7
Name: Dependency rate, dtype: int64

In [41]:
test_df['dependency squared'].value_counts()

1.000000     5388
0.000000     4289
0.250000     3678
4.000000     1769
2.250000     1758
0.111111     1533
0.444444     1130
64.000000    1037
0.062500      684
9.000000      596
1.777778      278
6.250000      224
0.040000      216
0.562500      203
16.000000     195
0.160000      175
0.360000      128
2.777778      120
25.000000      96
0.027778       56
1.562500       54
0.640000       45
0.020408       32
5.444444       30
0.694444       22
12.250000      18
49.000000      16
11.111111      13
0.734694       13
5.062500       13
0.140625       11
1.440000       11
0.081633        9
0.015625        9
36.000000       7
Name: dependency squared, dtype: int64

In [42]:
test_df['Dependency rate'] = np.sqrt(test_df['dependency squared'])

#### years of education of male head of household

In [43]:
# For male household heads
test_df[['years of education of male head of household', 'years of education of female head of household','edjefe squared']][test_df['years of education of male head of household']=='yes']

,years of education of male head of household,years of education of female head of household,edjefe squared
1272,yes,no,1
1273,yes,no,1
1463,yes,no,1
1464,yes,no,1
1857,yes,no,1
...,...,...,...
23731,yes,no,1
23796,yes,no,1
23797,yes,no,1
23798,yes,no,1


In [44]:
# For Female household heads
test_df[['years of education of male head of household', 'years of education of female head of household','years of schooling','=1 if household head']][test_df['years of education of female head of household']=='yes']

,years of education of male head of household,years of education of female head of household,years of schooling,=1 if household head
362,no,yes,0,0
363,no,yes,6,0
364,no,yes,6,0
365,no,yes,1,1
366,no,yes,5,0
...,...,...,...,...
22878,no,yes,1,1
23065,no,yes,1,1
23201,no,yes,11,0
23202,no,yes,1,1


In [45]:
# For Female household heads
test_df[['years of education of male head of household', 'years of education of female head of household','years of schooling','=1 if household head']][test_df['years of education of female head of household']=='yes'][test_df['=1 if household head']==1]

/var/folders/fj/pkqhb62j0v35sdfyvpr9kd940000gn/T/ipykernel_97690/3470362382.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  test_df[['years of education of male head of household', 'years of education of female head of household','years of schooling','=1 if household head']][test_df['years of education of female head of household']=='yes'][test_df['=1 if household head']==1]


,years of education of male head of household,years of education of female head of household,years of schooling,=1 if household head
365,no,yes,1,1
1116,no,yes,1,1
2510,no,yes,1,1
2726,no,yes,1,1
3000,no,yes,1,1
4413,no,yes,1,1
4434,no,yes,1,1
6804,no,yes,1,1
7296,no,yes,1,1
9112,no,yes,1,1


edjefe squared is the squared value of 'years of education of male head of household' and it is equals to 0 when the household head is female. Likewise, 'years of education of female head of household' id no when household head is male

* 'edjefe' and 'edjefa' are both 'no' when the head of the household had 0 years of school
* there's 'edjefe'= 'yes' and 'edjefa'='no' in some cases, all these cases the head of the household had 1 year of school
* there's 'edjefe'= 'no' and 'edjefa'='yes' in some cases, all these cases the head of the household had 1 year of school
* most of the time either 'edjefe' or 'edjefa' is a number while the other is a 'no'

In [46]:
# Merge to form a single column "Years of education of head of household"
new_col_values = []
for row in range(len(test_df)):
    male_value = test_df.iloc[row,:]['years of education of male head of household']
    female_value = test_df.iloc[row,:]['years of education of female head of household']
    if (male_value =='no' and female_value=='no'): # Both No
        new_col_values.append(0)
    elif (male_value =='yes' and female_value=='no'): # one yes
        new_col_values.append(1)
    elif (male_value =='no' and female_value=='yes'): # one yes
        new_col_values.append(1)
    elif (male_value !='no' and male_value !='yes' and female_value=='no'): # male is a number
        new_col_values.append(male_value)
    else:
        new_col_values.append(female_value)# female is a number

In [47]:
test_df["Years of education of head of household"] = new_col_values

In [48]:
test_df[['years of education of male head of household', 'years of education of female head of household','Years of education of head of household']]

,years of education of male head of household,years of education of female head of household,Years of education of head of household
0,no,17,17
1,no,17,17
2,no,17,17
3,16,no,16
4,no,11,11
...,...,...,...
23851,5,no,5
23852,6,no,6
23853,6,no,6
23854,6,no,6


In [49]:
test_df['Years of education of head of household'].value_counts()

6     7567
11    2707
9     1576
8     1173
3     1120
15    1093
0     1045
7      945
5      910
14     860
16     755
17     735
2      686
4      631
12     536
10     512
1      438
13     380
21      92
19      40
20      29
18      26
Name: Years of education of head of household, dtype: int64

In [50]:
test_df.drop(['years of education of male head of household', 'years of education of female head of household'], axis=1, inplace=True)

In [51]:
test_df.head()

,Household level identifier,Id,Monthly rent payment,=1 Overcrowding by bedrooms,number of all rooms in the house,=1 Overcrowding by rooms,=1 has toilet in the household,=1 if the household has refrigerator,owns a tablet,number of tablets household owns,...,escolari squared,age squared,hogar_total squared,edjefe squared,hogar_nin squared,overcrowding squared,dependency squared,meaned squared,Age squared,Years of education of head of household
0,72958b30c,ID_2f6873615,0.0,0,5,0,1,1,0,0.0,...,0,16,9,0,1,2.25,0.25,272.25,16,17
1,72958b30c,ID_1c78846d2,0.0,0,5,0,1,1,0,0.0,...,256,1681,9,0,1,2.25,0.25,272.25,1681,17
2,72958b30c,ID_e5442cf6a,0.0,0,5,0,1,1,0,0.0,...,289,1681,9,0,1,2.25,0.25,272.25,1681,17
3,5b598fbc9,ID_a8db26a79,0.0,0,14,0,1,1,1,1.0,...,256,3481,1,256,0,1.00,0.00,256.00,3481,16
4,1e2fc704e,ID_a62966799,175000.0,0,4,0,1,1,1,1.0,...,121,324,1,0,1,0.25,64.00,121.00,324,11


In [53]:
# test_df = pd.read_csv('test_df_renamed_new.csv')

### Converting one hot ecoded variables into numeric

In [56]:
test_df.head()

,Household level identifier,Id,Monthly rent payment,=1 Overcrowding by bedrooms,number of all rooms in the house,=1 Overcrowding by rooms,=1 has toilet in the household,=1 if the household has refrigerator,owns a tablet,number of tablets household owns,...,escolari squared,age squared,hogar_total squared,edjefe squared,hogar_nin squared,overcrowding squared,dependency squared,meaned squared,Age squared,Years of education of head of household
0,72958b30c,ID_2f6873615,0.0,0,5,0,1,1,0,0.0,...,0,16,9,0,1,2.25,0.25,272.25,16,17
1,72958b30c,ID_1c78846d2,0.0,0,5,0,1,1,0,0.0,...,256,1681,9,0,1,2.25,0.25,272.25,1681,17
2,72958b30c,ID_e5442cf6a,0.0,0,5,0,1,1,0,0.0,...,289,1681,9,0,1,2.25,0.25,272.25,1681,17
3,5b598fbc9,ID_a8db26a79,0.0,0,14,0,1,1,1,1.0,...,256,3481,1,256,0,1.00,0.00,256.00,3481,16
4,1e2fc704e,ID_a62966799,175000.0,0,4,0,1,1,1,1.0,...,121,324,1,0,1,0.25,64.00,121.00,324,11


In [57]:
def get_numeric(one_df):
    new_col = []
    for row in range(len(one_df)):
        current_row = one_df.iloc[row,:]
        for col in range(len(current_row)):
            if current_row[col] == 1:
                new_col.append(col+1)
                break
            elif col == (len(current_row)-1):
                print("not_found")
                new_col.append(0)
    return new_col

#### Walls

In [58]:
walls = ["=1 if walls are bad","=1 if walls are regular","=1 if walls are good"]

In [59]:
test_df['wall_condition'] = get_numeric(test_df[walls])

In [60]:
test_df = test_df.drop(walls,axis = 1)

In [61]:
test_df.head()

,Household level identifier,Id,Monthly rent payment,=1 Overcrowding by bedrooms,number of all rooms in the house,=1 Overcrowding by rooms,=1 has toilet in the household,=1 if the household has refrigerator,owns a tablet,number of tablets household owns,...,age squared,hogar_total squared,edjefe squared,hogar_nin squared,overcrowding squared,dependency squared,meaned squared,Age squared,Years of education of head of household,wall_condition
0,72958b30c,ID_2f6873615,0.0,0,5,0,1,1,0,0.0,...,16,9,0,1,2.25,0.25,272.25,16,17,3
1,72958b30c,ID_1c78846d2,0.0,0,5,0,1,1,0,0.0,...,1681,9,0,1,2.25,0.25,272.25,1681,17,3
2,72958b30c,ID_e5442cf6a,0.0,0,5,0,1,1,0,0.0,...,1681,9,0,1,2.25,0.25,272.25,1681,17,3
3,5b598fbc9,ID_a8db26a79,0.0,0,14,0,1,1,1,1.0,...,3481,1,256,0,1.00,0.00,256.00,3481,16,3
4,1e2fc704e,ID_a62966799,175000.0,0,4,0,1,1,1,1.0,...,324,1,0,1,0.25,64.00,121.00,324,11,2


#### Roofs

In [62]:
roof = ["=1 if roof are bad","=1 if roof are regular","=1 if roof are good"]

In [63]:
test_df['roof_condition'] = get_numeric(test_df[roof])

In [64]:
test_df = test_df.drop(roof,axis = 1)

In [65]:
test_df.head()

,Household level identifier,Id,Monthly rent payment,=1 Overcrowding by bedrooms,number of all rooms in the house,=1 Overcrowding by rooms,=1 has toilet in the household,=1 if the household has refrigerator,owns a tablet,number of tablets household owns,...,hogar_total squared,edjefe squared,hogar_nin squared,overcrowding squared,dependency squared,meaned squared,Age squared,Years of education of head of household,wall_condition,roof_condition
0,72958b30c,ID_2f6873615,0.0,0,5,0,1,1,0,0.0,...,9,0,1,2.25,0.25,272.25,16,17,3,3
1,72958b30c,ID_1c78846d2,0.0,0,5,0,1,1,0,0.0,...,9,0,1,2.25,0.25,272.25,1681,17,3,3
2,72958b30c,ID_e5442cf6a,0.0,0,5,0,1,1,0,0.0,...,9,0,1,2.25,0.25,272.25,1681,17,3,3
3,5b598fbc9,ID_a8db26a79,0.0,0,14,0,1,1,1,1.0,...,1,256,0,1.00,0.00,256.00,3481,16,3,3
4,1e2fc704e,ID_a62966799,175000.0,0,4,0,1,1,1,1.0,...,1,0,1,0.25,64.00,121.00,324,11,2,3


#### floors

In [66]:
floor = ["=1 if floor are bad","=1 if floor are regular","=1 if floor are good"]

In [67]:
test_df['floor_condition'] = get_numeric(test_df[floor])

In [68]:
test_df = test_df.drop(floor,axis = 1)

In [69]:
test_df.head()

,Household level identifier,Id,Monthly rent payment,=1 Overcrowding by bedrooms,number of all rooms in the house,=1 Overcrowding by rooms,=1 has toilet in the household,=1 if the household has refrigerator,owns a tablet,number of tablets household owns,...,edjefe squared,hogar_nin squared,overcrowding squared,dependency squared,meaned squared,Age squared,Years of education of head of household,wall_condition,roof_condition,floor_condition
0,72958b30c,ID_2f6873615,0.0,0,5,0,1,1,0,0.0,...,0,1,2.25,0.25,272.25,16,17,3,3,3
1,72958b30c,ID_1c78846d2,0.0,0,5,0,1,1,0,0.0,...,0,1,2.25,0.25,272.25,1681,17,3,3,3
2,72958b30c,ID_e5442cf6a,0.0,0,5,0,1,1,0,0.0,...,0,1,2.25,0.25,272.25,1681,17,3,3,3
3,5b598fbc9,ID_a8db26a79,0.0,0,14,0,1,1,1,1.0,...,256,0,1.00,0.00,256.00,3481,16,3,3,3
4,1e2fc704e,ID_a62966799,175000.0,0,4,0,1,1,1,1.0,...,0,1,0.25,64.00,121.00,324,11,2,3,3


#### Education level

In [70]:
edu_level = ["instlevel{}".format(i) for i in range(1,10)]
edu_level = list(np.ravel(feature_description.loc[edu_level].values))

In [71]:
test_df[edu_level]

,=1 no level of education,=1 incomplete primary,=1 complete primary,=1 incomplete academic secondary level,=1 complete academic secondary level,=1 incomplete technical secondary level,=1 complete technical secondary level,=1 undergraduate and higher education,=1 postgraduate higher education
0,1,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,1,0
2,0,0,0,0,0,0,0,0,1
3,0,0,0,0,0,0,0,1,0
4,0,0,0,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...
23851,0,1,0,0,0,0,0,0,0
23852,0,0,1,0,0,0,0,0,0
23853,0,1,0,0,0,0,0,0,0
23854,0,1,0,0,0,0,0,0,0


In [72]:
test_df['education_level'] = get_numeric(test_df[edu_level])

not_found
not_found
not_found
not_found
not_found


In [73]:
test_df = test_df.drop(edu_level,axis = 1)

In [74]:
test_df.head()

,Household level identifier,Id,Monthly rent payment,=1 Overcrowding by bedrooms,number of all rooms in the house,=1 Overcrowding by rooms,=1 has toilet in the household,=1 if the household has refrigerator,owns a tablet,number of tablets household owns,...,hogar_nin squared,overcrowding squared,dependency squared,meaned squared,Age squared,Years of education of head of household,wall_condition,roof_condition,floor_condition,education_level
0,72958b30c,ID_2f6873615,0.0,0,5,0,1,1,0,0.0,...,1,2.25,0.25,272.25,16,17,3,3,3,1
1,72958b30c,ID_1c78846d2,0.0,0,5,0,1,1,0,0.0,...,1,2.25,0.25,272.25,1681,17,3,3,3,8
2,72958b30c,ID_e5442cf6a,0.0,0,5,0,1,1,0,0.0,...,1,2.25,0.25,272.25,1681,17,3,3,3,9
3,5b598fbc9,ID_a8db26a79,0.0,0,14,0,1,1,1,1.0,...,0,1.00,0.00,256.00,3481,16,3,3,3,8
4,1e2fc704e,ID_a62966799,175000.0,0,4,0,1,1,1,1.0,...,1,0.25,64.00,121.00,324,11,2,3,3,5


### Removing Redundant

Delete needless columns

**redundant columns**

* r4t3, tamviv, tamhog, hhsize ... almost the same as hogar_total
* v14a ... almost the same as saniatrio1
* v18q, mobilephone ... can be generated by v18q1, qmobilephone
 SQBxxx, agesq ... squared values
* parentescoxxx ... only heads of household are in dataset now

**extra columns**

(One-hot variables should be linearly independent. For example, female (or male) column is needless, because whether the sample is female or not can be explained only with male (or female) column.)

* paredother, pisoother, abastaguano, energcocinar1, techootro, sanitario6, elimbasu6, estadocivil7, parentesco12, tipovivi5, lugar1, area1, female

**obsolete columns**
* epared1~3, etecho1~3, eviv1~3, instlevel1~9 ... we don't use these columns anymore.

In [75]:
needless_cols = ['r4t3', 'tamhog', 'tamviv', 'hhsize', 'v18q', 'v14a', 'agesq',
                 'mobilephone', 'paredother', 'pisoother', 'abastaguano',
                 'energcocinar1', 'techootro', 'sanitario6', 'elimbasu6',
                 'estadocivil7', 'parentesco12', 'tipovivi5',
                 'lugar1', 'area1', 'female']
SQB_cols = [s for s in test_df.columns.tolist() if 'squared' in s]
needless_cols = list(np.ravel(feature_description.loc[needless_cols].values))
needless_cols = needless_cols + SQB_cols

In [76]:
needless_cols

['Total persons in the household',
 'size of the household',
 'TamViv',
 'household size',
 'owns a tablet',
 '=1 has toilet in the household',
 'Age squared',
 '=1 if mobile phone',
 '=1 if predominant material on the outside wall is other',
 '=1 if predominant material on the floor is other',
 '=1 if no water provision',
 '=1 no main source of energy used for cooking (no kitchen)',
 '=1 if predominant material on the roof is other',
 '=1 toilet connected to other system',
 '=1 if rubbish disposal mainly other',
 '=1 if single',
 '=1 if other non family member',
 '=1 other(assigned, borrowed)',
 '=1 region Central',
 '=1 zona urbana',
 '=1 if female',
 'escolari squared',
 'age squared',
 'hogar_total squared',
 'edjefe squared',
 'hogar_nin squared',
 'overcrowding squared',
 'dependency squared',
 'meaned squared',
 'Age squared']

In [77]:
test_df = test_df.drop(needless_cols,axis = 1)

In [78]:
test_df.head()

,Household level identifier,Id,Monthly rent payment,=1 Overcrowding by bedrooms,number of all rooms in the house,=1 Overcrowding by rooms,=1 if the household has refrigerator,number of tablets household owns,Males younger than 12 years of age,Males 12 years of age and older,...,=1 region Brunca,=1 region Huetar AtlÃƒÂ¡ntica,=1 region Huetar Norte,=2 zona rural,Age in years,Years of education of head of household,wall_condition,roof_condition,floor_condition,education_level
0,72958b30c,ID_2f6873615,0.0,0,5,0,1,0.0,1,1,...,0,0,0,0,4,17,3,3,3,1
1,72958b30c,ID_1c78846d2,0.0,0,5,0,1,0.0,1,1,...,0,0,0,0,41,17,3,3,3,8
2,72958b30c,ID_e5442cf6a,0.0,0,5,0,1,0.0,1,1,...,0,0,0,0,41,17,3,3,3,9
3,5b598fbc9,ID_a8db26a79,0.0,0,14,0,1,1.0,0,1,...,0,0,0,0,59,16,3,3,3,8
4,1e2fc704e,ID_a62966799,175000.0,0,4,0,1,1.0,0,0,...,0,0,0,0,18,11,2,3,3,5


In [79]:
column_headers = list(test_df.columns)
# column_headers.remove('Target')
# column_headers.append('Target')
test_df=test_df[column_headers]
test_df.to_csv('test_df_renamed_new.csv', index=False)

### Others

In [ ]:
id = ['Id', 'Household level identifier', 'Target']

Dropping Column with no variance

In [ ]:
test_df['=1 if rubbish disposal mainly by throwing in river, creek or sea'].unique()

In [ ]:
test_df = test_df.drop(columns='=1 if rubbish disposal mainly by throwing in river, creek or sea')

In [ ]:
test_df_less_id = test_df.drop(columns = id)
# test_df_less_id.dropna()

In [ ]:
constant_filter = VarianceThreshold(threshold=0.01)
constant_filter.fit(test_df_less_id)

len(test_df_less_id.columns[constant_filter.get_support()])

qconstant_filter = [column for column in test_df_less_id.columns
                    if column not in test_df_less_id.columns[constant_filter.get_support()]]

qconstant_filter

In [ ]:
possible_removals = qconstant_filter


In [ ]:
len(qconstant_filter)

In [ ]:
fig, ax = plt.subplots(5, 5,figsize=(15,20))
j=0
for i in tq.tqdm(range(len(qconstant_filter))):
    if ((i+1)/5)>j+1:
        j+=1
#     print(f"j is {j} and i is {i} and division is {i%5}")
    ax[j,i%5].hist(test_df[qconstant_filter[i]])
    ax[j,i%5].set_title(qconstant_filter[i])
fig.tight_layout()

In [ ]:
no_variance = []
for i in range(len(test_df.columns)):
    value_list = test_df.iloc[:,i].unique()
    if len(value_list)<2:
        no_variance.append(test_df.iloc[:,i].name)

In [ ]:
# no_variance
test_df.head()

In [657]:
test_df.to_csv('test_df_renamed_new.csv', index=False)

## Sorting features

In [ ]:
# test_df = pd.read_csv("test_df_renamed.csv")

In [ ]:
test_df.head()

In [ ]:
id = ['Id', 'Household level identifier', 'Target']

In [ ]:
ind_bool = ['owns a tablet', '=1 if disable person', '=1 if male', '=1 if female', '=1 if less than 10 years old', '=1 if free or coupled uunion', '=1 if married', '=1 if divorced', '=1 if separated', '=1 if widow/er', '=1 if single', '=1 if household head', '=1 if spouse/partner', '=1 if son/doughter', '=1 if stepson/doughter', '=1 if son/doughter in law', '=1 if grandson/doughter', '=1 if mother/father', '=1 if father/mother in law', '=1 if brother/sister', '=1 if brother/sister in law', '=1 if other family member', '=1 if other non family member', '=1 no level of education', '=1 incomplete primary', '=1 complete primary', '=1 incomplete academic secondary level', '=1 complete academic secondary level', '=1 incomplete technical secondary level', '=1 complete technical secondary level', '=1 undergraduate and higher education', '=1 postgraduate higher education', '=1 if mobile phone']

ind_ordered = ['years of schooling', 'Age in years']


In [ ]:
hh_bool = ['=1 Overcrowding by bedrooms', '=1 Overcrowding by rooms', '=1 has toilet in the household', '=1 if the household has refrigerator', '=1 if predominant material on the outside wall is block or brick', '=1 if predominant material on the outside wall is socket (wood, zinc or absbesto', '=1 if predominant material on the outside wall is prefabricated or cement', '=1 if predominant material on the floor is cement', '=1 if predominant material on the outside wall is waste material', '=1 if predominant material on the outside wall is wood ', '=1 if predominant material on the outside wall is zink', '=1 if predominant material on the outside wall is natural fibers', '=1 if predominant material on the outside wall is other', '=1 if predominant material on the floor is mosaic, ceramic, terrazo', '=1 if predominant material on the floor is other', '=1 if predominant material on the floor is  natural material', '=1 if no floor at the household', '=1 if predominant material on the floor is wood', '=1 if predominant material on the roof is metal foil or zink', '=1 if predominant material on the roof is fiber cement, mezzanine ', '=1 if predominant material on the roof is natural fibers', '=1 if predominant material on the roof is other', '=1 if the house has ceiling', '=1 if water provision inside the dwelling', '=1 if water provision outside the dwelling', '=1 if no water provision', '=1 electricity from CNFL, ICE, ESPH/JASEC', '=1 electricity from private plant', '=1 no electricity in the dwelling', '=1 electricity from cooperative', '=1 no toilet in the dwelling', '=1 toilet connected to sewer or cesspool', '=1 toilet connected to  septic tank', '=1 toilet connected to black hole or letrine', '=1 toilet connected to other system', '=1 no main source of energy used for cooking (no kitchen)', '=1 main source of energy used for cooking electricity', '=1 main source of energy used for cooking gas', '=1 main source of energy used for cooking wood charcoal', '=1 if rubbish disposal mainly by tanker truck', '=1 if rubbish disposal mainly by botan hollow or buried', '=1 if rubbish disposal mainly by burning', '=1 if rubbish disposal mainly by throwing in an unoccupied space', '=1 if rubbish disposal mainly other', '=1 if walls are bad', '=1 if walls are regular', '=1 if walls are good', '=1 if roof are bad', '=1 if roof are regular', '=1 if roof are good', '=1 if floor are bad', '=1 if floor are regular', '=1 if floor are good', '=1 own and fully paid house', '=1 own, paying in installments', '=1 rented', '=1 precarious', '=1 other(assigned, borrowed)', '=1 if the household has notebook or desktop computer', '=1 if the household has TV', '=1 region Central', '=1 region Chorotega', '=1 region PacÃƒÂ\xadfico central', '=1 region Brunca', '=1 region Huetar AtlÃƒÂ¡ntica', '=1 region Huetar Norte', '=1 zona urbana', '=2 zona rural']

hh_ordered = [' number of all rooms in the house', 'Males younger than 12 years of age', 'Males 12 years of age and older', 'Total males in the household', 'Females younger than 12 years of age', 'Females 12 years of age and older', 'Total females in the household', 'persons younger than 12 years of age', 'persons 12 years of age and older', 'Total persons in the household', 'number of tablets household owns', 'size of the household', 'TamViv', 'household size', 'Number of children 0 to 19 in household', 'Number of adults in household', '# of individuals 65+ in the household', '# of total individuals in the household', 'number of bedrooms', '# of mobile phones']

hh_cont = ['Monthly rent payment', 'Dependency rate', 'years of education of male head of household', 'years of education of female head of household', 'average years of education for adults (18+)', '# persons per room']

In [ ]:
sqr_values = ['escolari squared', 'age squared', 'hogar_total squared', 'edjefe squared', 'hogar_nin squared', 'overcrowding squared', 'dependency squared', 'meaned squared', 'Age squared']

In [ ]:
sum_col = len(id)+len(ind_bool)+len(ind_ordered)+len(hh_bool)+len(hh_ordered)+len(hh_cont)+len(sqr_values)
print(f"Len of test is {len(test_df.columns)}")
print(f"Len of added columns is {sum_col}")

In [ ]:
ind_bool_df = test_df[ind_bool]
ind_ordered_df = test_df[ind_ordered]
hh_bool_df = test_df[hh_bool]
hh_ordered_df = test_df[hh_ordered]
hh_cont_df = test_df[hh_cont]
sqr_values = test_df[sqr_values]

#### Individual level

In [ ]:
# Set up the matplotlib plot configuration
#
f, ax = plt.subplots(figsize=(32, 24))
#
# Generate a mask for upper testgle
#
mask = np.triu(np.ones_like(ind_bool_df.corr(), dtype=bool))
#
# Configure a custom diverging colormap
#
cmap = sns.diverging_palette(230, 20, as_cmap=True)
#
# Draw the heatmap
plt.title("Correlation Heatmap for Individual level data",fontsize=24)
#
sns.heatmap(ind_bool_df.corr(), annot=True, mask = mask, cmap=cmap)

In [ ]:
household_df = test_df.loc[test_df['=1 if household head'] == 1, :]
household_df = household_df[ind_bool + ind_ordered]
household_df.shape

In [ ]:
# Create correlation matrix
corr_matrix = household_df.corr()

# Select upper triangle of correlation matrix
upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(np.bool))

# Find index of feature columns with correlation greater than 0.95
highly_correlated = [column for column in upper.columns if any(abs(upper[column]) > 0.95)]

highly_correlated

In [ ]:
# Information is directly inverse of that of 'if male'
to_drop = ['=1 if female']
possible_removals = possible_removals + to_drop

#### HH level variables

In [ ]:
household_df = test_df.loc[test_df['=1 if household head'] == 1, :]
household_df = household_df[hh_bool + hh_cont + hh_ordered]
household_df.shape

In [ ]:
# Create correlation matrix
corr_matrix = household_df.corr()

# Select upper triangle of correlation matrix
upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(np.bool))

# Find index of feature columns with correlation greater than 0.95
highly_correlated = [column for column in upper.columns if any(abs(upper[column]) > 0.95)]

highly_correlated

In [ ]:

# Set up the matplotlib plot configuration
#
f, ax = plt.subplots(figsize=(12, 6))
#
# Generate a mask for upper testgle
#
mask = np.triu(np.ones_like(test_df[highly_correlated].corr(), dtype=bool))
#
# Configure a custom diverging colormap
#
cmap = sns.diverging_palette(230, 20, as_cmap=True)
#
# Draw the heatmap
#
sns.heatmap(test_df[highly_correlated].corr(), annot=True, cmap=cmap)

In [ ]:
to_drop = ['household size', '# of total individuals in the household']
possible_removals = possible_removals + to_drop

### Possible removals

In [ ]:
set(possible_removals)

# Machine Learning Testing

In [1029]:
test_df = pd.read_csv("test_df_renamed_new.csv")
test_df = test_df.drop(columns = ['Household level identifier','Id'])
test_df.head()

,Monthly rent payment,=1 Overcrowding by bedrooms,number of all rooms in the house,=1 Overcrowding by rooms,=1 if the household has refrigerator,number of tablets household owns,Males younger than 12 years of age,Males 12 years of age and older,Total males in the household,Females younger than 12 years of age,...,=1 region Huetar AtlÃƒÂ¡ntica,=1 region Huetar Norte,=2 zona rural,Age in years,Years of education of head of household,wall_condition,roof_condition,floor_condition,education_level,Target
0,190000.0,0,3,0,1,0.0,0,1,1,0,...,0,0,0,43,10,2,1,1,4,4
1,135000.0,0,4,0,1,1.0,0,1,1,0,...,0,0,0,67,12,2,2,2,8,4
2,0.0,0,8,0,1,0.0,0,0,0,0,...,0,0,0,92,11,2,3,3,5,4
3,180000.0,0,5,0,1,1.0,0,2,2,1,...,0,0,0,17,11,3,3,3,4,4
4,180000.0,0,5,0,1,1.0,0,2,2,1,...,0,0,0,37,11,3,3,3,5,4


## Helper Functions

In [1030]:
#need to normalise some of the columns
def prepData(df,test_size = 0.3,normalize = True,include_eval = False):
    unnormal_cols = selectUnnormalised(df)
    x = df.iloc[:, :-1]
    y = df.iloc[:, -1:] 
    xtest, xtest, ytest, ytest = tts(x,y,test_size = test_size, random_state = 42)
    if include_eval:
        xtest, x_eval, ytest, y_eval = tts(xtest,ytest,test_size = 0.1/0.3, random_state = 42)
        return xtest.values, xtest.values, x_eval.values, ytest, ytest, y_eval
    else:
        if normalize:
            xtest, xTest = normalise(unnormal_cols, xtest, xtest)
            ytest, yTest = ytest.values.ravel(), ytest.values.ravel()
            return xtest, xTest, ytest, yTest
        return xtest.values, xtest.values, ytest, ytest
    
    
    

#getting the columns with non-normalised values
def selectUnnormalised(df):
    normCol = df.columns[df.isin([0,1]).all()] #getting the binary columns
    dfCols_list = list(df.columns)
    unnormCols = list(set(dfCols_list)-set(normCol))
    
    #remove target because we shouldn't normalise that, but rather encode
    unnormCols.remove('Target')
    return unnormCols

#normalising data in testing set
def normalise(unnormCols, xtest, xTest):
    #normalise testing data
    toBeNorm_test = xtest[[i for i in unnormCols]]
    ss = StandardScaler()
    std_scale = ss.fit(toBeNorm_test)
    xtest_norm = std_scale.transform(toBeNorm_test)
    
    #covert numpy array to df
    xtest_normCol = pd.DataFrame(xtest_norm, index = toBeNorm_test.index,
                                 columns = toBeNorm_test.columns)
    xtest.update(xtest_normCol)
    
    #normalise test data using mean and SD of testing set
    toBeNorm_test = xTest[[i for i in unnormCols]]
    xTest_norm = std_scale.transform(toBeNorm_test)
    xTest_normCol = pd.DataFrame(xTest_norm, index = toBeNorm_test.index,
                                columns = toBeNorm_test.columns)
    xTest.update(xTest_normCol)
    
    return xtest, xTest



In [465]:
# xtest, xtest, x_eval, ytest, ytest, y_eval = prepData(test_df,normalize=False,include_eval = True)

In [466]:
# datasets = [xtest, xtest, x_eval, ytest, ytest, y_eval]
# for i in datasets:
#     print(f"Len is {len(i)}")

Len is 6689
Len is 1911
Len is 957
Len is 6689
Len is 1911
Len is 957


In [869]:
# xtest, xtest, ytest, ytest = prepData(test_df,normalize=False)
xtest, xtest, ytest, ytest = prepData(test_df,test_size = 0.1)

In [853]:
# xtest, xTest, ytest, yTest = prepData(test_df,test_size = 0.1)

## LightGBM

In [859]:
# gbm = lgb.LGBMClassifier(boosting_type='dart', objective='multiclassova', class_weight='balanced', random_state=0)

# gbm.fit(xtest, ytest)
# y_pred = gbm.predict(xtest)

In [872]:
# build the model
lgb_clf = lgb.LGBMClassifier(max_depth=-1, learning_rate=0.03, objective='multiclass',
                             random_state=None, silent=True, metric='multi_logloss', 
                             n_jobs=4, n_estimators=5000, class_weight='balanced',
                             colsample_bytree =  0.89, min_child_samples = 90, num_leaves = 56, subsample = 0.96)

# fit data into the model and predict the test set
lgb_clf.fit(xtest.values, ytest, eval_set=[(xtest.values, ytest)], 
            early_stopping_rounds=400, verbose=100)
y_pred = lgb_clf.predict(xtest)

/usr/local/lib/python3.8/site-packages/lightgbm/sklearn.py:598: UserWarning: 'silent' argument is deprecated and will be removed in a future release of LightGBM. Pass 'verbose' parameter via keyword arguments instead.
  _log_warning("'silent' argument is deprecated and will be removed in a future release of LightGBM. "
/usr/local/lib/python3.8/site-packages/lightgbm/sklearn.py:726: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
/usr/local/lib/python3.8/site-packages/lightgbm/sklearn.py:736: UserWarning: 'verbose' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose' argument is deprecated and will be removed in a future releas

[100]	valid_0's multi_logloss: 0.556086
[200]	valid_0's multi_logloss: 0.341397
[300]	valid_0's multi_logloss: 0.23718
[400]	valid_0's multi_logloss: 0.180337
[500]	valid_0's multi_logloss: 0.151489
[600]	valid_0's multi_logloss: 0.136319
[700]	valid_0's multi_logloss: 0.1319
[800]	valid_0's multi_logloss: 0.13262
[900]	valid_0's multi_logloss: 0.136766
[1000]	valid_0's multi_logloss: 0.142303


In [876]:
cm = confusion_matrix(ytest, y_pred)
f1 = f1_score(ytest, y_pred, average='macro')
print("confusion matrix: \n", cm)
# print("macro F1 score: \n", f1)
print("Classification accuracy: {:.2f}".format(lgb_clf.score(xtest, ytest)))
print("F1 score : {:.2f}".format(f1_score(ytest, y_pred, average = 'macro')))

confusion matrix: 
 [[ 77   1   0   1]
 [  0 148   3   5]
 [  0   2 120   9]
 [  0   7   5 578]]
Classification accuracy: 0.97
F1 score : 0.96


In [878]:
final_df = copy.deepcopy(xtest)
final_df['Target'] = ytest
final_df['Pred'] = y_pred
final_df

,Monthly rent payment,=1 Overcrowding by bedrooms,number of all rooms in the house,=1 Overcrowding by rooms,=1 has toilet in the household,=1 if the household has refrigerator,owns a tablet,number of tablets household owns,Males younger than 12 years of age,Males 12 years of age and older,...,hogar_total squared,edjefe squared,hogar_nin squared,overcrowding squared,dependency squared,meaned squared,Age squared,Years of education of head of household,Target,Pred
8503,-0.421354,0,-0.652043,0,1,1,0,-0.467602,-0.571187,-1.506516,...,-0.967108,-0.679713,-0.555631,-0.722963,-0.312299,-0.708219,1.116875,-0.450839,1,1
7752,-0.421354,0,-0.652043,0,1,1,0,-0.467602,-0.571187,0.426580,...,0.894454,-0.565009,1.760820,1.383382,0.007169,1.988735,-0.667071,-1.133406,0,0
9350,-0.421354,0,-0.652043,0,1,1,0,-0.467602,0.901028,0.426580,...,-0.169296,-0.628734,0.023482,-0.355189,-0.232432,-0.961392,-0.897793,-1.360929,0,0
8414,-0.421354,0,0.026590,0,1,1,0,-0.467602,0.901028,-0.539968,...,-0.541608,-0.679713,-0.410853,-0.542420,-0.292333,-0.708219,0.164216,-0.450839,2,2
7671,-0.421354,0,0.026590,0,1,1,0,-0.467602,0.901028,-0.539968,...,0.894454,0.862416,3.063824,0.179756,1.684378,3.609040,-0.886916,0.686773,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
292,-0.421354,0,1.383856,0,1,1,0,-0.467602,-0.571187,1.393128,...,0.894454,0.594774,-0.410853,0.179756,-0.309105,0.293385,-0.428906,0.459251,3,3
1982,2.215327,0,0.026590,0,1,1,1,3.823125,-0.571187,0.426580,...,-0.169296,1.818282,0.023482,-0.355189,-0.232432,1.149268,-0.815352,1.369341,3,3
4387,0.285800,0,-1.330676,0,1,1,0,-0.467602,0.901028,-0.539968,...,-0.169296,0.135959,0.023482,0.179756,-0.232432,-0.228524,-0.357915,0.004206,3,3
5489,-0.421354,0,0.026590,0,1,1,0,-0.467602,-0.571187,0.426580,...,-0.169296,-0.220898,-0.555631,-0.355189,-0.312299,-0.492356,1.116875,-0.450839,3,3


In [880]:
final_hh_df = final_df[final_df["=1 if household head"]==1]
# print("Classification accuracy: {:.2f}".format(lgb_clf.score(xtest, ytest)))
print("F1 score : {:.2f}".format(f1_score(final_hh_df['Target'].values, final_hh_df['Pred'].values, average = 'macro')))

F1 score : 0.90


### Normalized

In [438]:
# build the model
lgb_clf = lgb.LGBMClassifier(max_depth=-1, learning_rate=0.03, objective='multiclass',
                             random_state=None, silent=True, metric='multi_logloss', 
                             n_jobs=4, n_estimators=5000, class_weight='balanced',
                             colsample_bytree =  0.89, min_child_samples = 90, num_leaves = 56, subsample = 0.96)

# fit data into the model and predict the test set
lgb_clf.fit(xtest, ytest, eval_set=[(xTest, yTest)], 
            early_stopping_rounds=400, verbose=100)
y_pred_norm = lgb_clf.predict(xTest)

/usr/local/lib/python3.8/site-packages/lightgbm/sklearn.py:598: UserWarning: 'silent' argument is deprecated and will be removed in a future release of LightGBM. Pass 'verbose' parameter via keyword arguments instead.
  _log_warning("'silent' argument is deprecated and will be removed in a future release of LightGBM. "
/usr/local/lib/python3.8/site-packages/lightgbm/sklearn.py:726: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
/usr/local/lib/python3.8/site-packages/lightgbm/sklearn.py:736: UserWarning: 'verbose' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose' argument is deprecated and will be removed in a future releas

[100]	valid_0's multi_logloss: 0.647179
[200]	valid_0's multi_logloss: 0.461691
[300]	valid_0's multi_logloss: 0.375939
[400]	valid_0's multi_logloss: 0.331828
[500]	valid_0's multi_logloss: 0.308643
[600]	valid_0's multi_logloss: 0.29832
[700]	valid_0's multi_logloss: 0.297571
[800]	valid_0's multi_logloss: 0.30052
[900]	valid_0's multi_logloss: 0.309504
[1000]	valid_0's multi_logloss: 0.319724


In [439]:
print("Classification accuracy: {:.2f}".format(lgb_clf.score(xTest, yTest)))
print("F1 score : {:.2f}".format(f1_score(yTest, y_pred_norm, average = 'macro')))

Classification accuracy: 0.92
F1 score : 0.88


### Just household heads

In [1031]:
test_hh_df = test_df[test_df["=1 if household head"]==1]

In [1033]:
test_hh_df = test_hh_df.drop("=1 if household head",axis =1)

In [1035]:
test_hh_df.head()

,Monthly rent payment,=1 Overcrowding by bedrooms,number of all rooms in the house,=1 Overcrowding by rooms,=1 if the household has refrigerator,number of tablets household owns,Males younger than 12 years of age,Males 12 years of age and older,Total males in the household,Females younger than 12 years of age,...,=1 region Huetar AtlÃƒÂ¡ntica,=1 region Huetar Norte,=2 zona rural,Age in years,Years of education of head of household,wall_condition,roof_condition,floor_condition,education_level,Target
0,190000.0,0,3,0,1,0.0,0,1,1,0,...,0,0,0,43,10,2,1,1,4,4
1,135000.0,0,4,0,1,1.0,0,1,1,0,...,0,0,0,67,12,2,2,2,8,4
2,0.0,0,8,0,1,0.0,0,0,0,0,...,0,0,0,92,11,2,3,3,5,4
5,180000.0,0,5,0,1,1.0,0,2,2,1,...,0,0,0,38,11,3,3,3,5,4
8,130000.0,1,2,0,1,0.0,0,1,1,2,...,0,0,0,30,9,1,1,2,4,4


In [1036]:
xtest, xTest, ytest, yTest = prepData(test_hh_df,test_size = 0.3)

In [1037]:
len(yTest)

892

In [1038]:
# build the model
lgb_clf = lgb.LGBMClassifier(max_depth=-1, learning_rate=0.03, objective='multiclass',
                             random_state=None, silent=True, metric='multi_logloss', 
                             n_jobs=4, n_estimators=5000, class_weight='balanced',
                             colsample_bytree =  0.89, min_child_samples = 90, num_leaves = 56, subsample = 0.96)

# fit data into the model and predict the test set
lgb_clf.fit(xtest.values, ytest, eval_set=[(xTest.values, yTest)], 
            early_stopping_rounds=400, verbose=100)
yPred = lgb_clf.predict(xTest)

/usr/local/lib/python3.8/site-packages/lightgbm/sklearn.py:598: UserWarning: 'silent' argument is deprecated and will be removed in a future release of LightGBM. Pass 'verbose' parameter via keyword arguments instead.
  _log_warning("'silent' argument is deprecated and will be removed in a future release of LightGBM. "
/usr/local/lib/python3.8/site-packages/lightgbm/sklearn.py:726: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
/usr/local/lib/python3.8/site-packages/lightgbm/sklearn.py:736: UserWarning: 'verbose' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose' argument is deprecated and will be removed in a future releas

[100]	valid_0's multi_logloss: 1.03757
[200]	valid_0's multi_logloss: 1.0127
[300]	valid_0's multi_logloss: 0.999112
[400]	valid_0's multi_logloss: 0.993283
[500]	valid_0's multi_logloss: 1.0049
[600]	valid_0's multi_logloss: 1.02145
[700]	valid_0's multi_logloss: 1.04227


In [1039]:
cm = confusion_matrix(yTest, yPred)
f1 = f1_score(yTest, yPred, average='macro')
print("confusion matrix: \n", cm)
print("macro F1 score: \n", f1)
# print("Classification accuracy: {:.2f}".format(lgb_clf.score(xtest, ytest)))
# print("F1 score : {}".format(f1_score(ytest.values, y_pred, average = 'macro')))

confusion matrix: 
 [[ 29  19   9  11]
 [ 27  54  29  30]
 [ 13  23  29  43]
 [ 27  60  65 424]]
macro F1 score: 
 0.43561947274878154
